# Scale Space

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# Read the image
orig_image = cv2.imread('book.png')
#cv2.imshow("book", orig_image)

for i in range(3):
    img = orig_image
    row, col, channel = img.shape
    img = cv2.resize(img, (row // (2**i), col // (2**i)))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    result = gray
    print(img.shape)
    
    for j in range(5):        
        smoothed_img = cv2.GaussianBlur(gray, (5,5), 2**j, 2**j)   
        result = cv2.hconcat([result, smoothed_img])         
    #cv2.imshow("result", result)
    
for i in range(3):
    img = orig_image
    row, col,channel = img.shape
    img = cv2.resize(img, (row // (2**i), col // (2**i)))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    result = gray
    print(img.shape)
    
    for j in range(5):        
        smoothed_img = cv2.GaussianBlur(gray, (5, 5), 2**j, 2**j) 
        dst = cv2.cornerHarris(smoothed_img, 3, 3, 0.04)
        dst = cv2.dilate(dst, None)
        smoothed_img[dst > 0.2 * dst.max()] = 255       
        result = cv2.hconcat([result, smoothed_img])
    
    cv2.imshow("result2", result)

(220, 264, 3)
(110, 132, 3)
(55, 66, 3)
(220, 264, 3)
(110, 132, 3)
(55, 66, 3)


# First Order Features

In [1]:
import numpy as np
import math

In [2]:
#Mean (u)
#Tells about the average intensity of the image
def mean(matrix):
    m,n = matrix.shape
    mean = 0
    for i in range(m):
        for j in range(n):
            mean = mean + matrix[i,j]
    mean = mean / (m*n)
    return mean

In [3]:
#Median (med)
#Gives the middle value of intensity after sorting it
def median(matrix):
    m,n = matrix.shape
    N = m * n
    matrix = matrix.flatten()
    matrix = np.sort(matrix)
    index = (N + 1)/2
    index = math.floor(index) - 1
    return matrix[index]

In [4]:
#Variance (sigma)^2
#Tells how much variation in intensites exist
#Variance indicates how likely gray level values vary within the image patch I
def variance(matrix):
    m,n = matrix.shape
    N = m * n
    variance = 0
    u = mean(matrix)
    for i in range(m):
        for j in range(n):
            variance = variance + ((matrix[i,j] - u) ** 2)
    variance = variance / (N - 1)
    return variance

In [5]:
#Standard deviation is computed by taking square root of the variance
def standard_deviation(matrix):
    return math.sqrt(variance(matrix))

In [6]:
def skewness(matrix):
    m,n = matrix.shape
    N = m * n
    u = mean(matrix)
    sigma = standard_deviation(matrix)
    sk = 0
    for i in range(m):
        for j in range(n):
            sk = sk + ((matrix[i,j] - u) ** 3)
    sk = sk/((N-1)*(sigma**3))
    return sk

In [7]:
def kurtosis(matrix):
    m,n = matrix.shape
    N = m * n
    u = mean(matrix)
    sigma = standard_deviation(matrix)
    sk = 0
    for i in range(m):
        for j in range(n):
            sk = sk + ((matrix[i,j] - u) ** 4)
    sk = sk/((N-1)*(sigma**4))
    return sk

In [8]:
#Mean Absolute Deviation (mad)
#Tells about the average difference of each intensity with the mean of image
def mean_absolute_deviation(matrix):
    m,n = matrix.shape
    N = m*n
    u = mean(matrix)
    mad = 0
    for i in range(m):
        for j in range(n):
            mad = mad + (matrix[i,j] - u)
    mad = mad / N
    return mad

In [9]:
def median_absolute_deviation(matrix):
    m,n = matrix.shape
    N = m*n
    Y = np.zeros((N))
    a = 0
    mad = mean_absolute_deviation(matrix)
    matrix = matrix.flatten()
    for i in range(N):
            Y[a] = matrix[i] - mad
            a = a + 1
    Y = np.sort(Y)
    print(Y)
    index = math.floor((N+1)/2)-1
    return Y[index]

In [10]:
def local_contrast(matrix):
    return np.max(matrix) - np.min(matrix)

In [11]:
def local_probability(matrix, k):
    m,n = matrix.shape
    N = m*n
    lp = 0
    for i in range(m):
        for j in range(n):
            if(matrix[i,j] == k):
                lp = lp + 1
    return lp/N

In [12]:
def percentile(matrix, x):
    m,n = matrix.shape
    N = m*n
    matrix = matrix.flatten()
    matrix = np.sort(matrix)
    index = N * (x/100)
    index = math.floor(index)-1
    return matrix[index]

# Second Order Features

In [30]:
def GLCM(matrix, d, theeta):
    minV = np.min(matrix)
    maxV = np.max(matrix)
    size = maxV - minV + 1
    M = np.zeros((size,size))
    m,n = matrix.shape
    for i in range(m):
        for j in range(n):
            if(theeta == 0):
                I = i
                J = j + d
            elif(theeta == 45):
                I = i - d
                J = j + d
            elif(theeta == 90):
                I = i - d
                J = j
            elif(theeta == 135):
                I = i - d
                J = j - d
            if(I < 0 or J < 0 or I >= m or J >= n):
                continue
            R = matrix[i,j]
            C = matrix[I,J]
            M[R,C] = M[R,C] + 1
    #print(M)        
    Mt = np.transpose(M)
    #print(Mt)
    Ms = M + Mt
    #print(Ms)
    sumV = Ms.sum()
    Ms = Ms / sumV
    return Ms

In [31]:
def energy(GLCM):
    return (GLCM ** 2).sum()

In [32]:
def homogenity(GLCM):
    m,n = GLCM.shape
    hom = 0
    for i in range(m):
        for j in range(n):
            hom = hom + (GLCM[i,j]/(1+((i-j)**2)))
    return hom

In [33]:
def contrast(GLCM):
    m,n = GLCM.shape
    hom = 0
    for i in range(m):
        for j in range(n):
            hom = hom + (((i-j)**2)*GLCM[i,j])
    return hom

In [34]:
def entropy(GLCM):
    m,n = GLCM.shape
    hom = 0
    for i in range(m):
        for j in range(n):
            hom = hom + (GLCM[i,j]*(math.log1p(GLCM[i,j])))
    return hom

In [35]:
def correlation(GLCM):
    ux = Ux(GLCM)
    uy = Uy(GLCM)
    sigmaX,sigmaY = Sigma(GLCM, ux)
    corr = 0
    m,n = GLCM.shape
    for i in range(m):
        for j in range(n):
            value = ((i-ux)*(j-uy)*GLCM[i,j])/(sigmaX*sigmaY)
            corr = corr + value        
    return corr
        
def Ux(GLCM):
    m,n = GLCM.shape
    ux = 0
    for i in range(m):
        ux = ux + Px(i,GLCM[i,:])
    return ux

def Uy(GLCM):
    return Ux(GLCM)

def Sigma(GLCM, u):
    m,n = GLCM.shape
    sigmaX = 0
    for i in range(m):
        sigmaX = sigmaX + ((Px(1,GLCM[i,:]))*((i-u)**2))
    sigmaX = math.sqrt(sigmaX)
    return sigmaX,sigmaX

def Px(i,row):
    return i * row.sum()

In [36]:
#matrix = np.array(([2,4],[6,8]))
#matrix = np.array(([0,0,1,1],[0,0,1,1],[0,2,2,2],[2,2,3,3]))
#matrix = np.array(([122,200,13],[41,55,216],[150,0,19]))
#matrix = np.array(([255,255,255],[0,0,0],[120,150,180]))
matrix = np.array(([5,11,7],[11,6,6],[9,8,7]))
print("Mean: ", mean(matrix))
print("Median: ", median(matrix))
print("Variance: ", variance(matrix))
print("Standard Deviation: ", standard_deviation(matrix))
print("Skewness: ", skewness(matrix))
print("Kurtosis: ", kurtosis(matrix))
print("Mean Absolute Deviation: ", mean_absolute_deviation(matrix))
print("Median Absolute Deviation: ", median_absolute_deviation(matrix))
print("Local Contrast: ", local_contrast(matrix))
print("Local Probability: ", local_probability(matrix, 255))
print("Percentile at 25: ", percentile(matrix, 25))
print("Percentile at 75: ", percentile(matrix, 75))

Mean:  7.777777777777778
Median:  7
Variance:  4.694444444444444
Standard Deviation:  2.1666666666666665
Skewness:  0.4318009406766805
Kurtosis:  1.6907437881493417
Mean Absolute Deviation:  9.868649107779169e-17
[ 5.  6.  6.  7.  7.  8.  9. 11. 11.]
Median Absolute Deviation:  7.0
Local Contrast:  6
Local Probability:  0.0
Percentile at 25:  6
Percentile at 75:  8


In [37]:
matrix = np.array(([1,2],[3,4]))
GLCM = GLCM(matrix,1,0)
print("GLCM: ", GLCM)
print("Energy: ", energy(GLCM))
print("Homogenity: ", homogenity(GLCM))
print("Contrast: ", contrast(GLCM))
print("Entropy: ", entropy(GLCM))
print("Correlation: ", correlation(GLCM))

IndexError: index 4 is out of bounds for axis 1 with size 4

In [38]:
print(GLCM.shape)

AttributeError: 'function' object has no attribute 'shape'